 # CMU 10-405/605 Machine Learning with Large Datasets
## **Welcome to Recitation 3**
This notebook will walk you through Spark transformations and actions. We will also process large Wikipedia dump to find interesting insights about your favorite celebrities.

#### ** Exploratory Data Analysis on Wikipedia dump **

In [0]:
# Load the Wikipedia dataset
wikiRDD = sc.wholeTextFiles("/FileStore/tables/text/*/*")

# Get all the pages
pages = wikiRDD.flatMap(lambda x: x[1].split('</doc>'))


def get_title_and_content(content):
    """
      This method generates a tuple of (title, actual content) from a Wiki dump of each page
    """
    # Remove any leading or lagging space if present 
    content = content.strip()
    title, actual_content = '', ''
    try:
        if(content != ''):
            # Split the content on the basis of new line
            arr = content.split("\n",2)
            # Second line is the title
            title = arr[1]
            # Rest is the actual content
            actual_content = arr[2]
    except:
        title, actual_content = '', ''
    return (title, actual_content)
  
  
# Extract the title and actual content
pages = pages.map(get_title_and_content).cache()


import re
def check_if_person(content):
    """ 
      This method checks if a page is of a celebrity or not.
    """
    # Checking in first 150 characters
    content = content[:150]
    # Checking for format like 12 August 1993
    list1 = re.findall(r"[\d]{1,2} [ADFJMNOS]\w* [\d]{4}", content)
    # Checking for format like August 12, 1993
    list2 = re.findall(r"[ADFJMNOS]\w* [\d]{1,2}[,] [\d]{4}", content)
    if(len(list1)>0 or len(list2)>0):
        return True
    return False
  

# Get all the celebrities pages
celebrities = pages.filter(lambda x : check_if_person(x[1]))

In [0]:
print(pages.count())
print(celebrities.count())

5571559
1282990

What is the most common length of the names in the WIKIPEDIA, ie, get a list of tuples in the format (length of names, count of articles having names of this length)?

In [0]:
# What is the most common length of the name
pages.map(lambda x: (len(x[0]), x[0])).groupByKey().map(lambda x: (x[0], len(x[1]))).takeOrdered(10, lambda x: -x[1])

Out[20]: [(13, 331299),
 (14, 327562),
 (12, 308414),
 (15, 305427),
 (16, 279434),
 (11, 264279),
 (17, 252603),
 (18, 230541),
 (19, 211265),
 (10, 205491)]

In [0]:
# Comparison with reduceByKey
pages.map(lambda x: (len(x[0]), 1)).reduceByKey(lambda x, y: x + y).takeOrdered(10, lambda x: -x[1])

Out[21]: [(13, 331299),
 (14, 327562),
 (12, 308414),
 (15, 305427),
 (16, 279434),
 (11, 264279),
 (17, 252603),
 (18, 230541),
 (19, 211265),
 (10, 205491)]

In [0]:
# What is the most common length of the names of the celebrities
celebrities.map(lambda x: (len(x[0]), 1)).reduceByKey(lambda x, y: x + y).takeOrdered(10, lambda x: -x[1])

Out[23]: [(13, 128108),
 (14, 121684),
 (12, 118997),
 (15, 103374),
 (11, 91794),
 (16, 82966),
 (17, 65634),
 (10, 55458),
 (18, 51747),
 (19, 42911)]

Tag each celebrity with its profession, either as FILM DIRECTOR, COMPUTER SCIENTIST, POLITICIAN or OTHER

In [0]:
# Using RDDs
def tag_profession(celebrity):
    """
      Tag the profession of the celebrity by finding the profession name in the content.
    """
    if "film director" in celebrity[1]:
        profession = "FILM DIRECTOR"
    elif "computer scientist" in celebrity[1]:
        profession = "COMPUTER SCIENTIST"
    elif "politician" in celebrity[1]:
        profession = "POLITICIAN"
    else:
        profession = "OTHER"
        
    return (celebrity[0], profession)
        

celebs_with_profession_rdd = celebrities.map(tag_profession)

celebs_with_profession_rdd.collect()[:10]

Out[36]: [('Abraham Lincoln', 'POLITICIAN'),
 ('Ayn Rand', 'OTHER'),
 ('Alain Connes', 'OTHER'),
 ('Allan Dwan', 'OTHER'),
 ('Andre Agassi', 'OTHER'),
 ('Animal Farm', 'OTHER'),
 ('Aldous Huxley', 'POLITICIAN'),
 ('Andrei Tarkovsky', 'FILM DIRECTOR'),
 ('Arthur Schopenhauer', 'OTHER'),
 ('Albert Sidney Johnston', 'POLITICIAN')]

How Dataframes can help?

In [0]:
# Convert the celebs RDD to DataFrame
celebsDF = celebrities.toDF(["title", "content"])

In [0]:
# Display the DataFrame
celebsDF.printSchema()
celebsDF.show(10)

root
-- title: string (nullable = true)
-- content: string (nullable = true)

+--------------------+--------------------+
 title| content|
+--------------------+--------------------+
 Abraham Lincoln|
Abraham Lincoln ...|
 Ayn Rand|
Ayn Rand (; born...|
 Alain Connes|
Alain Connes (; ...|
 Allan Dwan|
Allan Dwan (3 Ap...|
 Andre Agassi|
Andre Kirk Agass...|
 Animal Farm|
Animal Farm is a...|
 Aldous Huxley|
Aldous Leonard H...|
 Andrei Tarkovsky|
Andrei Arsenyevi...|
 Arthur Schopenhauer|
Arthur Schopenha...|
Albert Sidney Joh...|
Albert Sidney Jo...|
+--------------------+--------------------+
only showing top 10 rows

In [0]:
# Create a new column as profession where a profession can be either FILM DIRECTOR, COMPUTER SCIENTIST or POLITICIAN
from pyspark.sql import functions as F

celebs_with_profession = celebsDF.withColumn("profession", 
                           F.when(F.col("content").contains("film director"), "FILM DIRECTOR")
                           .when(F.col("content").contains("computer scientist"), "COMPUTER SCIENTIST")
                           .when(F.col("content").contains("politician"), "POLITICIAN")
                           .when(F.col("content").contains("artist"), "ARTIST")
                           .otherwise("OTHER")).select("title", "profession")

In [0]:
celebs_with_profession.show(10)

+--------------------+-------------+
 title| profession|
+--------------------+-------------+
 Abraham Lincoln| POLITICIAN|
 Ayn Rand| ARTIST|
 Alain Connes| OTHER|
 Allan Dwan| OTHER|
 Andre Agassi| OTHER|
 Animal Farm| ARTIST|
 Aldous Huxley| POLITICIAN|
 Andrei Tarkovsky|FILM DIRECTOR|
 Arthur Schopenhauer| ARTIST|
Albert Sidney Joh...| POLITICIAN|
+--------------------+-------------+
only showing top 10 rows

In [0]:
celebs_with_profession.groupBy("profession").count().orderBy('count').show()

+------------------+-------+
 profession| count|
+------------------+-------+
COMPUTER SCIENTIST| 1036|
 FILM DIRECTOR| 10124|
 ARTIST| 84506|
 POLITICIAN| 116834|
 OTHER|1070490|
+------------------+-------+

Find the celebs that are related to museums

In [0]:
celebs_in_museums = celebsDF.filter(F.col("content").contains("museum"))

celebs_in_museums.show(10)

+--------------------+--------------------+
 title| content|
+--------------------+--------------------+
 Abraham Lincoln|
Abraham Lincoln ...|
 Andy Warhol|
Andy Warhol (; b...|
 Akira Kurosawa|
Akira Kurosawa (...|
 Albert Schweitzer|
Albert Schweitze...|
 Aarhus|
Aarhus (, , ; of...|
 Aleister Crowley|
Aleister Crowley...|
Alfred Russel Wal...|
Alfred Russel Wa...|
 Alexander Agassiz|
Alexander Emmanu...|
 Alessandro Volta|
Alessandro Giuse...|
 Abel Tasman|
Abel Janszoon Ta...|
+--------------------+--------------------+
only showing top 10 rows

Give a distribution of celebs featured in museum based on their profession, ie, count the number of celebs featured in museum by profession

In [0]:
celebs_in_museum_with_profession = celebs_in_museums.join(celebs_with_profession, celebs_in_museums.title == celebs_with_profession.title)\
                                                    .groupBy("profession").count().orderBy('count').show()

+------------------+-----+
 profession|count|
+------------------+-----+
COMPUTER SCIENTIST| 10|
 FILM DIRECTOR| 203|
 POLITICIAN| 1187|
 ARTIST| 5934|
 OTHER| 7914|
+------------------+-----+

Get the significance of dates in a Wiki articles for a celebrity by counting the number of times a month name appears in the article divided by the total number of paragraphs.

In [0]:
# Using RDDs 

In [0]:
def date_significance(celebrity):
    """
        Calculates significance of dates for a celebrity by counting the number of times a month name appears in the article divided by the total number of paragraphs.
    """
    lst_of_mnths = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    month_cnt = [celebrity[1].count(month) for month in lst_of_mnths]
    
    return (celebrity[0], float(sum(month_cnt))/len(celebrity[1].split('\n\n')))
   

celebrities_with_date_signficance_rdd = celebrities.map(date_significance)

In [0]:
celebrities_with_date_signficance_rdd.filter(lambda x: x[1] > 1).count()

Out[54]: 244114

In [0]:
# Using DataFrames

In [0]:
from pyspark.sql.functions import udf

@udf("float")
def date_significance_udf(content):
    """
        Calculates significance of dates for a celebrity by counting the number of times a month name appears in the article divided by the total number of paragraphs.
    """
    lst_of_mnths = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    month_cnt = [content.count(month) for month in lst_of_mnths]
    
    return float(sum(month_cnt))/len(content.split('\n\n'))
  
celebrities_with_date_signficance_df = celebsDF.withColumn('date_significance', date_significance_udf(celebsDF.content)).select("title", "date_significance")

In [0]:
celebrities_with_date_signficance_df.filter(celebrities_with_date_signficance_df.date_significance > 1).count()

Out[73]: 244114

Calculate the average date significance by profession only for those cases where date signifance > 1

In [0]:
# Using RDDs : CASE 1

In [0]:
celebrities_with_date_signficance_rdd.join(celebs_with_profession_rdd).filter(lambda x: x[1][0] > 1).collect()

Out[81]: [('Aleister Crowley', (1.1967213114754098, 'OTHER')),
 ('Al Capone', (1.0810810810810811, 'OTHER')),
 ('Anthony of Saxony', (1.5384615384615385, 'OTHER')),
 ('Barbara Olson', (1.1111111111111112, 'OTHER')),
 ('Buddy Holly', (1.16, 'OTHER')),
 ('Chuck Yeager', (1.025, 'OTHER')),
 ('Charles Edward Jones', (1.2, 'OTHER')),
 ('Coronation Street', (1.2352941176470589, 'OTHER')),
 ('Clara Petacci', (1.2, 'OTHER')),
 ('Dave Winer', (1.3214285714285714, 'OTHER')),
 ('David Bowie', (1.1018518518518519, 'OTHER')),
 ('Ernst Mayr', (2.391304347826087, 'OTHER')),
 ('Foreign relations of Estonia', (1.2727272727272727, 'OTHER')),
 ('Erwin Rommel', (1.3716216216216217, 'POLITICIAN')),
 ('Prince Eugene of Savoy', (1.7462686567164178, 'OTHER')),
 ('Engelbert Dollfuss', (1.2222222222222223, 'POLITICIAN')),
 ('Louis Mountbatten, 1st Earl Mountbatten of Burma',
 (1.1891891891891893, 'POLITICIAN')),
 ('Frank Sinatra', (1.3195876288659794, 'OTHER')),
 ('First Battle of El Alamein', (1.0833333333333333, 'OTHER')),
 ('Frederick I, Margrave of Brandenburg-Ansbach',
 (1.6666666666666667, 'OTHER')),
 ('Guy Fawkes', (1.5, 'OTHER')),
 ('Heinrich Himmler', (1.2105263157894737, 'OTHER')),
 ('Hermann Göring', (1.2575757575757576, 'OTHER')),
 ('Herman Melville', (1.2857142857142858, 'OTHER')),
 ('Prince Harry, Duke of Sussex', (1.5694444444444444, 'OTHER')),
 ('John Lennon', (1.1282051282051282, 'OTHER')),
 ('Josip Broz Tito', (1.6578947368421053, 'POLITICIAN')),
 ('Kosovo War', (1.0942028985507246, 'POLITICIAN')),
 ('Kylie Minogue', (1.7307692307692308, 'OTHER')),
 ('Kelsey Grammer', (1.05, 'POLITICIAN')),
 ('Karl Gustav Ahlefeldt', (2.0, 'OTHER')),
 ('Laura Bush', (1.1428571428571428, 'OTHER')),
 ('Leon Trotsky', (1.2483221476510067, 'POLITICIAN')),
 ('Lola Graham', (4.0, 'OTHER')),
 ('Louis Riel', (1.1428571428571428, 'POLITICIAN')),
 ('Mehmet Ali Ağca', (1.0833333333333333, 'OTHER')),
 ('Martin Bormann', (1.6842105263157894, 'OTHER')),
 ('Niels Bohr', (1.3409090909090908, 'OTHER')),
 ('Neville Chamberlain', (1.0510204081632653, 'POLITICIAN')),
 ('Pope John XXIII', (1.337837837837838, 'OTHER')),
 ('Pope Honorius IV', (1.0833333333333333, 'OTHER')),
 ('Pope Martin IV', (2.35, 'OTHER')),
 ('Diana, Princess of Wales', (1.0909090909090908, 'POLITICIAN')),
 ('Pope Celestine II', (1.375, 'OTHER')),
 ('Pope Gregory XI', (1.3846153846153846, 'OTHER')),
 ('Pope Urban VI', (1.1333333333333333, 'OTHER')),
 ('Victoria, Crown Princess of Sweden', (1.0555555555555556, 'OTHER')),
 ('Pope Innocent V', (2.0, 'OTHER')),
 ('Rudy Giuliani', (1.4017857142857142, 'POLITICIAN')),
 ('Afghan Civil War (1928–1929)', (2.610169491525424, 'OTHER')),
 ('Shania Twain', (1.0952380952380953, 'OTHER')),
 ('Dreamcast', (1.027027027027027, 'OTHER')),
 ('Second Battle of El Alamein', (1.0348837209302326, 'OTHER')),
 ('Theodore Judah', (1.1818181818181819, 'OTHER')),
 ('William Morris', (1.0246913580246915, 'OTHER')),
 ('Winter War', (1.0731707317073171, 'POLITICIAN')),
 ('Willi Hennig', (1.368421052631579, 'OTHER')),
 ('Wendell Willkie', (1.0434782608695652, 'POLITICIAN')),
 ('1570s', (2.0, 'OTHER')),
 ('1540s', (2.0, 'OTHER')),
 ('1000s (decade)', (2.0, 'OTHER')),
 ('1040s', (2.0, 'OTHER')),
 ('1680s', (2.0, 'OTHER')),
 ('1160s', (1.5, 'OTHER')),
 ('1600s (decade)', (2.0, 'OTHER')),
 ('1310s', (2.0, 'OTHER')),
 ('1320s', (1.5, 'OTHER')),
 ('1380s', (2.0, 'OTHER')),
 ('1510s', (2.0, 'OTHER')),
 ('1050s', (2.0, 'OTHER')),
 ('1140s', (2.0, 'OTHER')),
 ('1190s', (2.0, 'OTHER')),
 ('1550s', (2.0, 'OTHER')),
 ('1660s', (2.0, 'OTHER')),
 ('1030s', (2.0, 'OTHER')),
 ('1440s', (2.0, 'OTHER')),
 ('1410s', (2.0, 'OTHER')),
 ('1230s', (2.0, 'OTHER')),
 ('1290s', (2.0, 'OTHER')),
 ('Roger B. Chaffee', (1.0588235294117647, 'OTHER')),
 ('Denise Richards', (1.52, 'OTHER')),
 ('Oskar Schindler', (1.0789473684210527, 'OTHER')),
 ('Mike Tyson', (1.0804597701149425, 'FILM DIRECTOR')),
 ('Ernest Lawrence', (1.135135135135135, 'OTHER')),
 ('Richard Somers', (1.4, 'OTHER')),
 ('La Scala', (1.04761904761904

In [0]:
# Using RDDs : CASE 2

In [0]:
celebrities_with_date_signficance_rdd.filter(lambda x: x[1] > 1).join(celebs_with_profession_rdd).collect()

Out[76]: [('Aleister Crowley', (1.1967213114754098, 'OTHER')),
 ('Al Capone', (1.0810810810810811, 'OTHER')),
 ('Anthony of Saxony', (1.5384615384615385, 'OTHER')),
 ('Barbara Olson', (1.1111111111111112, 'OTHER')),
 ('Buddy Holly', (1.16, 'OTHER')),
 ('Chuck Yeager', (1.025, 'OTHER')),
 ('Charles Edward Jones', (1.2, 'OTHER')),
 ('Coronation Street', (1.2352941176470589, 'OTHER')),
 ('Clara Petacci', (1.2, 'OTHER')),
 ('Dave Winer', (1.3214285714285714, 'OTHER')),
 ('David Bowie', (1.1018518518518519, 'OTHER')),
 ('Ernst Mayr', (2.391304347826087, 'OTHER')),
 ('Foreign relations of Estonia', (1.2727272727272727, 'OTHER')),
 ('Erwin Rommel', (1.3716216216216217, 'POLITICIAN')),
 ('Prince Eugene of Savoy', (1.7462686567164178, 'OTHER')),
 ('Engelbert Dollfuss', (1.2222222222222223, 'POLITICIAN')),
 ('Louis Mountbatten, 1st Earl Mountbatten of Burma',
 (1.1891891891891893, 'POLITICIAN')),
 ('Frank Sinatra', (1.3195876288659794, 'OTHER')),
 ('First Battle of El Alamein', (1.0833333333333333, 'OTHER')),
 ('Frederick I, Margrave of Brandenburg-Ansbach',
 (1.6666666666666667, 'OTHER')),
 ('Guy Fawkes', (1.5, 'OTHER')),
 ('Heinrich Himmler', (1.2105263157894737, 'OTHER')),
 ('Hermann Göring', (1.2575757575757576, 'OTHER')),
 ('Herman Melville', (1.2857142857142858, 'OTHER')),
 ('Prince Harry, Duke of Sussex', (1.5694444444444444, 'OTHER')),
 ('John Lennon', (1.1282051282051282, 'OTHER')),
 ('Josip Broz Tito', (1.6578947368421053, 'POLITICIAN')),
 ('Kosovo War', (1.0942028985507246, 'POLITICIAN')),
 ('Kylie Minogue', (1.7307692307692308, 'OTHER')),
 ('Kelsey Grammer', (1.05, 'POLITICIAN')),
 ('Karl Gustav Ahlefeldt', (2.0, 'OTHER')),
 ('Laura Bush', (1.1428571428571428, 'OTHER')),
 ('Leon Trotsky', (1.2483221476510067, 'POLITICIAN')),
 ('Lola Graham', (4.0, 'OTHER')),
 ('Louis Riel', (1.1428571428571428, 'POLITICIAN')),
 ('Mehmet Ali Ağca', (1.0833333333333333, 'OTHER')),
 ('Martin Bormann', (1.6842105263157894, 'OTHER')),
 ('Niels Bohr', (1.3409090909090908, 'OTHER')),
 ('Neville Chamberlain', (1.0510204081632653, 'POLITICIAN')),
 ('Pope John XXIII', (1.337837837837838, 'OTHER')),
 ('Pope Honorius IV', (1.0833333333333333, 'OTHER')),
 ('Pope Martin IV', (2.35, 'OTHER')),
 ('Diana, Princess of Wales', (1.0909090909090908, 'POLITICIAN')),
 ('Pope Celestine II', (1.375, 'OTHER')),
 ('Pope Gregory XI', (1.3846153846153846, 'OTHER')),
 ('Pope Urban VI', (1.1333333333333333, 'OTHER')),
 ('Victoria, Crown Princess of Sweden', (1.0555555555555556, 'OTHER')),
 ('Pope Innocent V', (2.0, 'OTHER')),
 ('Rudy Giuliani', (1.4017857142857142, 'POLITICIAN')),
 ('Afghan Civil War (1928–1929)', (2.610169491525424, 'OTHER')),
 ('Shania Twain', (1.0952380952380953, 'OTHER')),
 ('Dreamcast', (1.027027027027027, 'OTHER')),
 ('Second Battle of El Alamein', (1.0348837209302326, 'OTHER')),
 ('Theodore Judah', (1.1818181818181819, 'OTHER')),
 ('William Morris', (1.0246913580246915, 'OTHER')),
 ('Winter War', (1.0731707317073171, 'POLITICIAN')),
 ('Willi Hennig', (1.368421052631579, 'OTHER')),
 ('Wendell Willkie', (1.0434782608695652, 'POLITICIAN')),
 ('1570s', (2.0, 'OTHER')),
 ('1540s', (2.0, 'OTHER')),
 ('1000s (decade)', (2.0, 'OTHER')),
 ('1040s', (2.0, 'OTHER')),
 ('1680s', (2.0, 'OTHER')),
 ('1160s', (1.5, 'OTHER')),
 ('1600s (decade)', (2.0, 'OTHER')),
 ('1310s', (2.0, 'OTHER')),
 ('1320s', (1.5, 'OTHER')),
 ('1380s', (2.0, 'OTHER')),
 ('1510s', (2.0, 'OTHER')),
 ('1050s', (2.0, 'OTHER')),
 ('1140s', (2.0, 'OTHER')),
 ('1190s', (2.0, 'OTHER')),
 ('1550s', (2.0, 'OTHER')),
 ('1660s', (2.0, 'OTHER')),
 ('1030s', (2.0, 'OTHER')),
 ('1440s', (2.0, 'OTHER')),
 ('1410s', (2.0, 'OTHER')),
 ('1230s', (2.0, 'OTHER')),
 ('1290s', (2.0, 'OTHER')),
 ('Roger B. Chaffee', (1.0588235294117647, 'OTHER')),
 ('Denise Richards', (1.52, 'OTHER')),
 ('Oskar Schindler', (1.0789473684210527, 'OTHER')),
 ('Mike Tyson', (1.0804597701149425, 'FILM DIRECTOR')),
 ('Ernest Lawrence', (1.135135135135135, 'OTHER')),
 ('Richard Somers', (1.4, 'OTHER')),
 ('La Scala', (1.04761904761904

In [0]:
# Using DATAFRAMES: CASE 1

In [0]:
celebrities_with_date_signficance_df.join(celebs_with_profession, celebrities_with_date_signficance_df.title == celebs_with_profession.title).filter(celebrities_with_date_signficance_df.date_significance > 1).show()

+--------------------+-----------------+--------------------+----------+
 title|date_significance| title|profession|
+--------------------+-----------------+--------------------+----------+
10th Kansas Volun...| 11.333333|10th Kansas Volun...| OTHER|
 1939 in Poland| 2.0| 1939 in Poland| OTHER|
 1946 Paris–Nice| 2.0| 1946 Paris–Nice| OTHER|
1959 Critérium du...| 2.0|1959 Critérium du...| OTHER|
1984 Australian O...| 2.0|1984 Australian O...| OTHER|
1985 Tour of the ...| 2.0|1985 Tour of the ...| OTHER|
1986 World Fencin...| 2.0|1986 World Fencin...| OTHER|
1987 Vuelta a Murcia| 2.0|1987 Vuelta a Murcia| OTHER|
1988 United State...| 1.5|1988 United State...| OTHER|
1995–96 Slovenian...| 2.0|1995–96 Slovenian...| OTHER|
1996 Albanian par...| 1.5|1996 Albanian par...| OTHER|
1999 IBF World Ch...| 2.0|1999 IBF World Ch...| OTHER|
 2 Months 2 Million| 1.3333334| 2 Months 2 Million| OTHER|
2001 Danish local...| 1.25|2001 Danish local...| OTHER|
2003 Asian Women'...| 2.0|2003 Asian Women'...| OTHER|
2004 World Team T...| 3.0|2004 World Team T...| OTHER|
2006 European Wom...| 2.0|2006 European Wom...| OTHER|
2006 Holland Ladi...| 2.0|2006 Holland Ladi...| OTHER|
2006 Las Vegas De...| 2.5|2006 Las Vegas De...| OTHER|
2006–07 Australia...| 2.6111112|2006–07 Australia...| OTHER|
+--------------------+-----------------+--------------------+----------+
only showing top 20 rows

In [0]:
# Using DATAFRAMES: CASE 2

In [0]:
celebrities_with_date_signficance_df.filter(celebrities_with_date_signficance_df.date_significance > 1).join(celebs_with_profession, celebrities_with_date_signficance_df.title == celebs_with_profession.title).show()

+--------------------+-----------------+--------------------+----------+
 title|date_significance| title|profession|
+--------------------+-----------------+--------------------+----------+
10th Kansas Volun...| 11.333333|10th Kansas Volun...| OTHER|
 1939 in Poland| 2.0| 1939 in Poland| OTHER|
 1946 Paris–Nice| 2.0| 1946 Paris–Nice| OTHER|
1959 Critérium du...| 2.0|1959 Critérium du...| OTHER|
1984 Australian O...| 2.0|1984 Australian O...| OTHER|
1985 Tour of the ...| 2.0|1985 Tour of the ...| OTHER|
1986 World Fencin...| 2.0|1986 World Fencin...| OTHER|
1987 Vuelta a Murcia| 2.0|1987 Vuelta a Murcia| OTHER|
1988 United State...| 1.5|1988 United State...| OTHER|
1995–96 Slovenian...| 2.0|1995–96 Slovenian...| OTHER|
1996 Albanian par...| 1.5|1996 Albanian par...| OTHER|
1999 IBF World Ch...| 2.0|1999 IBF World Ch...| OTHER|
 2 Months 2 Million| 1.3333334| 2 Months 2 Million| OTHER|
2001 Danish local...| 1.25|2001 Danish local...| OTHER|
2003 Asian Women'...| 2.0|2003 Asian Women'...| OTHER|
2004 World Team T...| 3.0|2004 World Team T...| OTHER|
2006 European Wom...| 2.0|2006 European Wom...| OTHER|
2006 Holland Ladi...| 2.0|2006 Holland Ladi...| OTHER|
2006 Las Vegas De...| 2.5|2006 Las Vegas De...| OTHER|
2006–07 Australia...| 2.6111112|2006–07 Australia...| OTHER|
+--------------------+-----------------+--------------------+----------+
only showing top 20 rows

In [0]:
celebrities_with_date_signficance_df.cache()

Out[83]: DataFrame[title: string, date_significance: float]

In [0]:
celebrities_with_date_signficance_df.filter(celebrities_with_date_signficance_df.date_significance > 1).join(celebs_with_profession, celebrities_with_date_signficance_df.title == celebs_with_profession.title).show()

+--------------------+-----------------+--------------------+----------+
 title|date_significance| title|profession|
+--------------------+-----------------+--------------------+----------+
10th Kansas Volun...| 11.333333|10th Kansas Volun...| OTHER|
 1939 in Poland| 2.0| 1939 in Poland| OTHER|
 1946 Paris–Nice| 2.0| 1946 Paris–Nice| OTHER|
1959 Critérium du...| 2.0|1959 Critérium du...| OTHER|
1984 Australian O...| 2.0|1984 Australian O...| OTHER|
1985 Tour of the ...| 2.0|1985 Tour of the ...| OTHER|
1986 World Fencin...| 2.0|1986 World Fencin...| OTHER|
1987 Vuelta a Murcia| 2.0|1987 Vuelta a Murcia| OTHER|
1988 United State...| 1.5|1988 United State...| OTHER|
1995–96 Slovenian...| 2.0|1995–96 Slovenian...| OTHER|
1996 Albanian par...| 1.5|1996 Albanian par...| OTHER|
1999 IBF World Ch...| 2.0|1999 IBF World Ch...| OTHER|
 2 Months 2 Million| 1.3333334| 2 Months 2 Million| OTHER|
2001 Danish local...| 1.25|2001 Danish local...| OTHER|
2003 Asian Women'...| 2.0|2003 Asian Women'...| OTHER|
2004 World Team T...| 3.0|2004 World Team T...| OTHER|
2006 European Wom...| 2.0|2006 European Wom...| OTHER|
2006 Holland Ladi...| 2.0|2006 Holland Ladi...| OTHER|
2006 Las Vegas De...| 2.5|2006 Las Vegas De...| OTHER|
2006–07 Australia...| 2.6111112|2006–07 Australia...| OTHER|
+--------------------+-----------------+--------------------+----------+
only showing top 20 rows

TODO: Find the average using groupBy() on profession

In [0]:
celebrities_with_date_signficance_df.filter(celebrities_with_date_signficance_df.date_significance == 0.0).count()

Out[89]: 665